# Different modes of VaspInteractive
In this tutorial you will see the context and classic modes of `VaspInteractive` and how they differ. 

We continue using the H2 example as in tutorial 01

In [2]:
%rm -rf sandbox/vpi-modes

In [3]:
%mkdir -p sandbox/vpi-modes

In [4]:
%env VASP_COMMAND

'mpirun -np 8 --map-by hwthread /opt/vasp.6.1.2_pgi_mkl_beef/bin/vasp_std'

In [6]:
from ase.atoms import Atoms
from ase.optimize import BFGS
h2 = Atoms("H2", positions=[(0.95, 0, 0), (0, 0, 0)], cell=[8, 8, 8], pbc=True)
print("Scaled positions of H2 in the cell:")
print(h2.get_scaled_positions())

Scaled positions of H2 in the cell:
[[0.11875 0.      0.     ]
 [0.      0.      0.     ]]


In [7]:
from vasp_interactive import VaspInteractive
vasp_params = dict(xc="pbe", kpts=1, encut=200)
calc = VaspInteractive(directory="sandbox/vpi-modes", **vasp_params)

### Classical mode
In this mode, the VASP process starts once the `calc.calculate` is invoked (e.g. `atoms.get_potential_energy()`). It needs to be stopped manually.

In [11]:
atoms = h2.copy()
atoms.calc = calc
print("Process is: ", calc.process)

Process is:  None


In [13]:
atoms.get_potential_energy()
print(calc.process)
if calc.process.poll() is None:
    print("Process is still running")

Process is still running


Let's check the OUTCAR, it should stop at the ionic step but no fields like `Total CPU time` is shown

In [19]:
%%bash
tail -20 sandbox/vpi-modes/OUTCAR




  FREE ENERGIE OF THE ION-ELECTRON SYSTEM (eV)
  ---------------------------------------------------
  free  energy   TOTEN  =        -6.19701990 eV

  energy  without entropy=       -6.20861571  energy(sigma->0) =       -6.20088517
 


--------------------------------------------------------------------------------------------------------


    POTLOK:  cpu time    0.0182: real time    0.0182


--------------------------------------------------------------------------------------------------------




Change the atoms positions and redo calculation

In [21]:
atoms.rattle()
atoms.get_potential_energy()
calc.process

assume we've finished relaxation, manually close the process, but results still stored on calculator

In [24]:
calc.finalize()
print(calc.process)
print(calc.get_potential_energy())

None
-6.20829497


### Context mode
Basically just use a with-clause to wrap the VaspInteractive region. Once outside it the process is automatically cleaned up

In [26]:
atoms = h2.copy()
with calc:
    atoms.calc = calc
    for i in range(3):
        atoms.rattle()
        print(atoms.get_potential_energy())

-6.20459881
-6.20827522
-6.2119499


In [28]:
print(calc.process)

None
